<a href="https://colab.research.google.com/github/rexian/ML/blob/main/agenticai/crewai/automate_event_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Key Elements of Tools:
* Versatility
* Fault Tolerance
* Caching

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install -U -q crewai crewai_tools langchain_community

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
import getpass
os.environ['NVIDIA_NIM_API_KEY'] = getpass.getpass("Enter API key for NVIDIA AI Endpoints: ")

Enter API key for NVIDIA AI Endpoints: ··········


In [5]:
os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter OPENAI API Key: ")
os.environ['SERPER_API_KEY'] = getpass.getpass("Enter Serper API Key: ")

Enter OPENAI API Key: ··········
Enter Serper API Key: ··········


In [6]:
from crewai.llm import LLM

llm = LLM(
    model="nvidia_nim/meta/llama3-70b-instruct",
    temperature=0.1
)

In [7]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [8]:
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    llm = llm,
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [9]:
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    llm = llm,
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [10]:
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    llm = llm,
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)


### Creating Venue Pydantic Object
* Create a class VenueDetails using Pydantic BaseModel.
* Agents will populate this object with information about different venues by creating different instances of it.

In [11]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks

* By using output_json, you can specify the structure of the output you want.
* By using output_file, you can get your output in a file.
* By setting human_input=True, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [12]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [13]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [14]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=False,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

In [15]:
event_management_crew = Crew(
    agents=[venue_coordinator, logistics_manager, marketing_communications_agent],
    tasks=[venue_task, logistics_task, marketing_task],
    verbose=True
)

## Running the Crew

In [16]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2025-06-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

Display final output

In [18]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '12 N. San Pedro St., Downtown San Jose, CA 95110',
 'booking_status': 'Bookable',
 'capacity': 8000,
 'name': 'Blanco Urban Venue'}


In [19]:
from IPython.display import Markdown
Markdown("marketing_report.md")

**Marketing Plan for Tech Innovation Conference**
==============================================

**Objective:** Engage at least 500 potential attendees for the Tech Innovation Conference on 2025-06-15.

**Target Audience:** Tech professionals, entrepreneurs, and innovators interested in the latest technology trends and innovations.

**Marketing Strategies:**

1. **Create a Strong Website and Event App:** Design a dedicated website and event app to provide essential information, registration, and engagement opportunities.
2. **Social Media Campaign:** Utilize Facebook, Twitter, LinkedIn, and Instagram to create buzz around the conference. Share engaging content, speaker profiles, and sponsorship opportunities.
3. **Influencer Marketing:** Partner with industry influencers and thought leaders to promote the conference and share their expertise.
4. **Email Marketing:** Design a series of email campaigns to inform, engage, and remind potential attendees about the conference.
5. **Content Marketing:** Develop a content calendar featuring blog posts, whitepapers, and case studies on relevant tech topics.
6. **Video Marketing:** Create promotional videos, speaker interviews, and conference highlights to showcase on the website and social media channels.
7. **Gamification and Incentives:** Offer early-bird discounts, referral incentives, and networking opportunities to motivate registrations.
8. **Paid Advertising:** Run targeted online ads on Google, Facebook, and LinkedIn to reach the target audience.
9. **Sponsorship and Partnerships:** Collaborate with tech companies and organizations to sponsor the conference and provide additional value to attendees.
10. **PR and Media Outreach:** Craft a press release announcing the conference and pitch stories to media outlets, tech blogs, and industry publications.

**Timeline:**

* 6 weeks prior to the conference: Launch website, social media campaign, and email marketing series.
* 4 weeks prior to the conference: Intensify social media and email marketing efforts, launch paid advertising campaigns.
* 2 weeks prior to the conference: Finalize sponsorship and partnership agreements, distribute press release.

**Budget Allocation:**

* Website and event app development: 20%
* Social media and email marketing: 30%
* Influencer marketing: 10%
* Content marketing: 10%
* Video marketing: 5%
* Gamification and incentives: 5%
* Paid advertising: 20%
* Sponsorship and partnerships: 5%
* PR and media outreach: 5%

By implementing these marketing strategies, we can effectively promote the Tech Innovation Conference and engage at least 500 potential attendees.